# SAX Encoding

Questo notebook è stato ispirato da un [articolo letto mesi fa su KDNuggets](https://www.kdnuggets.com/2019/09/time-series-baseball.html) in cui si parla del SAX Encoding per trovare Time Series anomale.

Nell'articolo viene usato un dataset sul baseball per trovare stagioni anomale per varie squadre, visto il momento in cui viene scritto provo a utilizzare il SAX Encoding sui [dati della protezione civile](https://github.com/pcm-dpc/COVID-19) per vedere se ci sono serie anomale tra i contagi da [*Covid-19*](https://it.wikipedia.org/wiki/COVID-19).

* L'orizzonte temporale và dal 24/02/2020 al 03/04/2020.
* La distribuzione geografica è per Regione anche se il Trentino - Alto Adige è diviso nelle due province autonome.

### ATTENZIONE!!! IMPORTANTE.
**Naturalmente non sono un medico/epidemiologo/virologo/... quindi il mio interesse era solo trovare un dataset che mi permettesse di spiegare come funziona l'algoritmo, non trarrò nessuna conclusione dai dati.**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

dati_regioni = pd.read_csv("./dpc-covid19-ita-regioni.csv")

In [ ]:
dati_regioni.head()

La colonna che useremo sarà *totale_positivi* che, riporto direttamente dal readme del [repository ufficiale della protezione civile](https://github.com/pcm-dpc/COVID-19)

> Totale attualmente positivi (ospedalizzati + isolamento domiciliare)

In [ ]:
totale_positivi = dati_regioni[["data", "denominazione_regione", "totale_positivi"]].copy()
totale_positivi.head()

## Cos'è il SAX Encoding?

Il SAX encoding è un metodo per semplificare le serie storiche, viene ridotta la dimensionalità col fine di trovare pattern anomali.

Può essere considerato un metodo non supervisionato di *anomaly detection* dove per anomalia (o outlier) non si intende la singola osservazione ma l'intera serie storica.

Le serie in questo caso sono tutte uguali, visto che non c'è assenza di dati, ma bisogna evidenziare che il SAX è una tecnica molto robusta ai valori mancanti e quindi utilizzabile quando si è in presenza di serie di lunghezza diversa, come nell'articolo sul baseball citato in precedenza.

### Step 1: Standardizzare la serie

* Per applicare questa tecnica bisogna avere la serie storica sulla riga e ad ogni colonna coinciderà uno step temporale.

* Le serie devono essere standardizzate così da avere la stessa scala.

In [ ]:
tot_pos_pivot = totale_positivi.pivot(index="data", columns="denominazione_regione", values="totale_positivi").reset_index()

In [ ]:
tot_pos_pivot.head()

In [ ]:
tot_pos_pivot.tail()

In [ ]:
tot_pos_pivot.plot()
#tot_pos_pivot.plot(legend=False)

In [ ]:
df_stand = ((tot_pos_pivot.drop("data", axis=1) - tot_pos_pivot.mean())/tot_pos_pivot.std()).T

In [ ]:
df_stand

Ora abbiamo la nostra matrice con sull'indice il nome della serie, ovvero la regione e sulle colonne il periodo da 0 a 39.

### Step 2: Piecewise Aggregate Approximation

Come accennato inzialmente questo metodo riduce la dimensionalità della serie e lo fa proprio con la P.A.A., per applicarla avremo bisogno di scegliere una finestra temporale, in questo caso visto che i vari ragionamenti (sentiti in TV, ribadisco di non avere nessuna competenza) si basano sulla settimana imposterò il parametro (window) *w=7*.

Ridimensionata ogni serie in serie di *linghezza_serie*/*w* periodi si prende la media del periodo come nuovo valore.

**Attenzione.** Bisogna impostare un controllo per non perdere informazioni, quindi **se il nuovo numero di periodi non è un intero và sempre arrotondato per eccesso.**

In [ ]:
windows = 7

new_len = int(np.ceil((df_stand.shape[1]/windows)))

# Crea il nuovo dataFrame
df_PAA = pd.DataFrame(index = df_stand.index, columns = range(0, new_len))

In [ ]:
# Calcola la media di ogni window
ind = 0
for i in range(0,df_stand.shape[1], windows):
    avg = df_stand.iloc[:,i:i+windows].mean(axis=1)
    df_PAA[ind] = avg.values
    ind +=1

In [ ]:
df_PAA.head()

### Step 3: SAX

Il core di questa tecnica è che ora il dato numerico viene converito in stringa secondo dei livelli. La scelta dei livelli influisce sul risultato quindi bisognerebbe sempre farsi affiancare da un esperto di dominio.

Io non conoscendo il fenomeno supporrò 3 livelli ["A", "B", "C"] e suddividerò i valori:
* dal minimo fino al primo quartile
* tra primo e terzo quartile
* superiore al terzo quartile

In questi caso potremmo interpretarli come:
* pochi contagi, ottima situazione
* situazione nella media
* grave emergenza

Una volta definiti i livelli questi vengono concatenati in un'unica stringa detta appunto "*SAX string*". Es. ABAAB

In [ ]:
# Sto ridondando con i DataFrame creati ma è per mostrare i vari step, sorry.
binned = pd.DataFrame(index = df_PAA.index, columns = df_PAA.columns)

for j in range(0, df_PAA.shape[1]):
            bins = []
            bins.append(df_PAA[j].min()-.01)
            bins.append(df_PAA[j].quantile([0.25]).values[0])
            bins.append(df_PAA[j].quantile([0.75]).values[0])
            bins.append(df_PAA[j].max()+.01)
            labels = ["A", "B", "C"]
            binned[j] = pd.cut(df_PAA[j], bins, labels=labels)
binned['sequence'] = binned.apply(''.join, axis=1)
            
binned

### Step 4: Quando un outlier è un outlier?

In questo caso torna indispensabile la presenza di un esperto di dominio, io non essendolo (e lo scriverò ogni volta) ho fatto delle prove fino a trovare un risultato che mi permetta di mostrare qualcosa.

Infatti una volta ottenute le SAX String bisogna contare le frequenze di ognuna e fissare una soglia per cui una serie è considerata anomala.

In [ ]:
# imposto il limite a 1
limit = 1

# serie originale trasposta
encoded = tot_pos_pivot.T
encoded.drop(index="data", inplace = True)

# aggiungo l'etichetta se è un outlier o no
freq = binned.sequence.value_counts()
encoded['outlier'] = binned['sequence'].isin(list(freq[freq<=limit].index))

In [ ]:
encoded.head()

In [ ]:
encoded["outlier"].value_counts()

Da questo esperimento, considerando una finestra di 7 giorni e che un outlier per noi è chi ha una sequenza unica (viste anche le sole 21 serie) abbiamo 6 regioni che hanno lo stesso andamento e 15 che si discostano da tutte le altre.

In [ ]:
plt.figure(figsize=(18,10))
for i in range(encoded.shape[0]):
    if encoded.iloc[i,-1]:
        col = 'r'
    else:
        col = 'b'
    plt.plot(encoded.iloc[i,:-1], col)
plt.legend()
plt.title("Totale positivi per Regione dopo SAX Encoding")

Ultima cosa da notare è che non sono solo i volumi ad influenzare questa tecnica ma anche gli andamenti come testimoniano le serie di Friuli e Lombardia che hanno volumi molto differenti ma probabilmente andamenti simili.

*Se volete provare altre soluzioni potete provare a variare la finestra temporale e la soglia per essere considerato outlier*.